In [ ]:
import os
import time

import pandas as pd
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_community.chat_models import ChatOllama

from deepeval import evaluate
from deepeval.metrics import FaithfulnessMetric, ContextualPrecisionMetric, AnswerRelevancyMetric, ContextualRelevancyMetric, ContextualRecallMetric
from deepeval.test_case import LLMTestCase
from warnings import filterwarnings
filterwarnings('ignore')

os.environ["OPENAI_API_KEY"] = ""

load_dotenv()

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index = pc.Index(os.environ["PINECONE_INDEX"])
print(index.describe_index_stats())

vectorstore = PineconeVectorStore(  
    index=index, 
    embedding=OpenAIEmbeddings(), 
    text_key="text"  
)

# base model
gpt3_5 = ChatOpenAI(      
    model_name='gpt-3.5-turbo',  
    temperature=0.1  
)

T3Q = ChatOllama(
    model="T3Q-ko-solar-dpo-v7.0:latest", 
    name="T3Q-ko-solar-dpo-v7.0", 
    temperature=0.1, 
    repeat_penalty=1.2,
    top_k=10,
    top_p=0.5
)

eeve = ChatOllama(
    model="yanoljaEEVE-Korean-Instruct-10.8B-v1.0:latest", 
    name="EEVE-Korean-Instruct-10.8B-v1.0", 
    temperature=0.1,        
    repeat_penalty=1.2,
    top_k=10,
    top_p=0.5
)

platypus = ChatOllama(
    model="KoR-Orca-Platypus-13B:latest", 
    name="KoR-Orca-Platypus-13B", 
    temperature=0.1,        
    repeat_penalty=1.2,
    top_k=10,
    top_p=0.5
)

llama3_bllossom = ChatOllama(
    model="llama-3-Korean-Bllossom-8B:latest", 
    name="llama-3-Korean-Bllossom-8B", 
    temperature=0.1,        
    repeat_penalty=1.2,
    top_k=10,
    top_p=0.5
)

llama3 = ChatOllama(
    model="Meta-Llama-3-8B:latest", 
    name="Meta-Llama-3-8B", 
    temperature=0.1,        
    repeat_penalty=1.2,
    top_k=10,
    top_p=0.5
)

def evaluate_faithfulness(llm_name, query, answer, **kwargs):
    context = kwargs.get('context')
    FAITHFUL = FaithfulnessMetric(
        threshold=0.7,
        model="gpt-4",
        include_reason=True
    )

    test_case = LLMTestCase(
        input=query,
        actual_output=answer,
        retrieval_context=context
    )

    score = -1
    reason = "error"
    try:
        FAITHFUL.measure(test_case)
        score = FAITHFUL.score
        reason = FAITHFUL.reason
        print('metric.score', score)
        print("-" * 50)
        print('metric.reason', reason)
        print("-" * 50)
    except Exception as e:
        print('e', e)
        pass

    test_result = {
        'llm_name': llm_name, 
        'input': query, 
        'actual_output': answer, 
        'context': '\n\n'.join(context), 
        'metric_score': score, 
        'metric_reason': reason
    }
    return test_result

def evaluate_contextual_precision(llm_name, query, answer, **kwargs):
    expected_output = kwargs.get('expected_output')
    context = kwargs.get('context')
    CONTEXTUAL_PRECISION = ContextualPrecisionMetric(
        threshold=0.7,
        model="gpt-4",
        include_reason=True
    )

    test_case = LLMTestCase(
        input=query,
        actual_output=answer,
        expected_output=expected_output,
        retrieval_context=context
    )

    score = -1
    reason = "error"
    try:
        CONTEXTUAL_PRECISION.measure(test_case)
        score = CONTEXTUAL_PRECISION.score
        reason = CONTEXTUAL_PRECISION.reason
        print('metric.score', score)
        print("-" * 50)
        print('metric.reason', reason)
        print("-" * 50)
    except Exception as e:
        print('e', e)
        pass

    test_result = {
        'llm_name': llm_name, 
        'input': query, 
        'actual_output': answer, 
        'context': '\n\n'.join(context), 
        'metric_score': score, 
        'metric_reason': reason
    }
    return test_result

def evaluate_contextual_relacancy(llm_name, query, answer, **kwargs):
    context = kwargs.get('context')
    CONTEXTUAL_RELAVANCY = ContextualRelevancyMetric(
        threshold=0.7,
        model="gpt-4",
        include_reason=True
    )

    test_case = LLMTestCase(
        input=query,
        actual_output=answer,
        retrieval_context=context
    )

    score = -1
    reason = "error"
    try:
        CONTEXTUAL_RELAVANCY.measure(test_case)
        score = CONTEXTUAL_RELAVANCY.score
        reason = CONTEXTUAL_RELAVANCY.reason
        print('metric.score', score)
        print("-" * 50)
        print('metric.reason', reason)
        print("-" * 50)
    except Exception as e:
        print('e', e)
        pass

    test_result = {
        'llm_name': llm_name, 
        'input': query, 
        'actual_output': answer, 
        'context': '\n\n'.join(context), 
        'metric_score': score, 
        'metric_reason': reason
    }
    return test_result

def evaluate_contextual_recall(llm_name, query, answer, expected_output, **kwargs):
    context = kwargs.get('context')
    CONTEXTUAL_RECALL = ContextualRecallMetric(
        threshold=0.7,
        model="gpt-4",
        include_reason=True
    )

    test_case = LLMTestCase(
        input=query,
        actual_output=answer,
        expected_output=expected_output,
        retrieval_context=context
    )
    score = -1
    reason = "error"
    try:
        CONTEXTUAL_RECALL.measure(test_case)
        score = CONTEXTUAL_RECALL.score
        reason = CONTEXTUAL_RECALL.reason
        print('metric.score', score)
        print("-" * 50)
        print('metric.reason', reason)
        print("-" * 50)
    except Exception as e:
        print('e', e)
        pass

    test_result = {
        'llm_name': llm_name, 
        'input': query, 
        'actual_output': answer, 
        'context': '\n\n'.join(context), 
        'metric_score': score, 
        'metric_reason': reason
    }
    return test_result

def evaluate_answer_relavancy(llm_name, query, answer, **kwargs):
    context = kwargs.get('context')
    ANSWER_RELAVANCY = AnswerRelevancyMetric(
        threshold=0.7,
        model="gpt-4",
        include_reason=True
    )

    test_case = LLMTestCase(
        input=query,
        actual_output=answer
    )
    score = -1
    reason = "error"
    try:
        ANSWER_RELAVANCY.measure(test_case)
        score = ANSWER_RELAVANCY.score
        reason = ANSWER_RELAVANCY.reason
        print('metric.score', score)
        print("-" * 50)
        print('metric.reason', reason)
        print("-" * 50)
    except Exception as e:
        print('e', e)
        pass

    test_result = {
        'llm_name': llm_name, 
        'input': query, 
        'actual_output': answer, 
        'context': '\n\n'.join(context), 
        'metric_score': score, 
        'metric_reason': reason
    }
    return test_result

llms = [
    # gpt3_5,
    T3Q,
    eeve,
    platypus,
    llama3_bllossom,
    llama3
]

metrics = [
    # {'label': 'FAITHFUL', 'value': evaluate_faithfulness},
    {'label': 'CONTEXTUAL_PRECISION', 'value': evaluate_contextual_precision},
    {'label': 'CONTEXTUAL_RELAVANCY', 'value': evaluate_contextual_relacancy},
    {'label': 'CONTEXTUAL_RECALL', 'value': evaluate_contextual_recall},
    {'label': 'ANSWER_RELAVANCY', 'value': evaluate_answer_relavancy}
]

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a friendly and polite machine learning expert chatbot.
Answer the user's question based on the [context] below.
Don't answer anything that isn't in the context and say, "I'm not sure how to answer that question, please be more specific."
Don't answer in one sentence, but in 3-5 sentences.
Give reasons and evidence.
All answers should be in Korean.\n\n[context]\n{context}"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

queries = [
    "온라인 학습 시스템이 무엇인가요?",
    "검증 오차가 상승하면 미니배치 경사 하강법을 즉시 중단하는 것이 좋은 방법인가요?",
    "릿지 회귀를 사용했을 때 훈련 오차와 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $a$를 증가시켜야 할까요, 아니면 줄여야 할까요?",
    "사진을 낮과 밤, 실내와 실외로 분류하려 합니다. 두 개의 로지스틱 회귀 분류기를 만들어야 할까요, 아니면 하나의 소프트맥스 회귀 분류기를 만들어야 할까요?",
    "수백만 개의 샘플과 수백 개의 특성을 가진 훈련 세트에 SVM 모델을 훈련시키려면 원 문제와 쌍대 문제 중 어떤 것을 사용해야 하나요?",
    "결정 트리가 훈련 세트에 과소적합되었다면 입력 특성의 스케일을 조정하는 것이 좋을까요?",
    "oob 평가의 장점은 무엇인가요?",
    "차원의 저주란 무엇인가요?",
    "적층 오토인코더의 하위층에서 학습한 특성을 시각화하기 위해 사용하는 일반적인 기법은 무엇인가요? 상위층에 대해서는 어떻게 할 수 있나요?",
    "강화 학습 에이전트의 성능은 어떻게 측정할 수 있나요?"
]

answers = [
    "온라인 학습 시스템은 배치 학습 시스템과 달리 점진적으로 학습할 수 있습니다. 이 방식은 변화하는 데이터와 자율 시스템에 빠르게 적응하고 매우 많은 양의 데이터를 훈련시킬 수 있습니다",
    "무작위성 때문에 확률적 경사 하강법이나 미니배치 경사 하강법 모두 매 훈련 반복마다 학습의 진전을 보장하지 못합니다. 검증 에러가 상승될 때 훈련을 즉시 멈춘다면 최적점에 도달하기 전에 너무 일찍 멈추게 될지 모릅니다. 더 나은 방법은 정기적으로 모델을 저장하고 오랫동안 진전이 없을 때 (즉, 최상의 점수를 넘어서지 못하면), 저장된 것 중 가장 좋은 모델로 복원하는 것입니다.",
    "훈련 에러와 검증 에러가 거의 비슷하고 매우 높다면 모델이 훈련 세트에 과소적합되었을 가능성이 높습니다. 즉, 높은 편향을 가진 모델입니다. 따라서 규제 하이퍼파라미터 $a$를 감소시켜야 합니다.",
    "실외와 실내, 낯과 밤에 따라 사진을 구분하고 싶다면 이 둘은 배타적인 클래스가 아니기 때문에(즉, 네 가지 조합이 모두 가능하므로) 두 개의 로지스틱 회귀 분류기를 훈련시켜야 합니다.",
    "커널 SVM은 쌍대 형식만 사용할 수 있기 때문에 이 질문은 선형 SVM에만 해당합니다. 원 문제의 계산 복잡도는 훈련 샘플 수에 비례하지만, 쌍대 형식의 계산 복잡도는 삼각대수에 비례합니다. 그러므로 수백만 개의 샘플이 있다면 쌍대 형식은 너무 느려질 것이므로 원 문제를 사용해야 합니다.",
    "결정 트리가 훈련 세트에 과대적합되었다면 모델에 제약을 가해 규제해야 하므로 max. depth를 낮추는 것이 좋습니다.",
    "oob 평가를 사용하면 배깅 앙상블의 각 예측기가 훈련에 포함되지 않은 (즉, 따로 떼어놓은) 샘플을 사용해 평가됩니다. 이는 추가적인 검증 세트가 없어도 편향되지 않게 앙상블을 평가하도록 도와줍니다. 그러므로 훈련에 더 많은 샘플을 사용할 수 있어서 앙상블의 성능은 조금 더 향상될 것입니다.",
    "우리는 3차원 세계에서 살고 있어서 고차원 공간을 직관적으로 상상하기 어렵습니다. 1,000차원의 공간에서 휘어져 있는 200차원의 타원체는 고사하고 기본적인 4차원 초입방체(hypercube)조차도 머릿속에 그리기 어렵습니다. 차원의 저주는 저차원 공간에는 없는 많은 문제가 고차원 공간에서 일어난다는 사실을 뜻합니다. 머신러닝에서 무작위로 선택한 고차원 벡터는 매우 희소해서 과대적합의 위험이 크고, 많은 양의 데이터가 있지 않으면 데이터에 있는 패턴을 잡아내기 매우 어려운 것이 흔한 현상입니다.",
    "스택 오토인코더의 하위층이 학습한 특성을 시각화하기 위한 일반적인 방법은 각 뉴런의 가중치를 입력 이미지의 크기로 바꾸어 그려보는 것입니다（예를 들어 MNIST에서는 [784] 크기의 가중치 벡터를 [28, 28]로 바꿈니다）. 상위층에서 학습한 특성을 시각화하기 위한 한 가지 방법은 각 뉴런을 가장 활성화시키는 훈련 샘플을 그려보는 것입니다.",
    "강화 학습 에이전트의 성능을 측정하려면 간단하게 얻은 보상을 모두 더하면 됩니다. 시물레이션 환경에서는 많은 에피소드를 수행하고 평균적으로 얻은 전체 보상을 확인합니다（최소，최대，표준 편차 등을 볼 수 있습니다）"
]

# 메모리 리스트(추가 예정)
chat_history = []

def chat_with_llm(llm, vectorstore, prompt, query):
    retriever_chain = create_history_aware_retriever(llm, vectorstore.as_retriever(search_kwargs={"k": 6}), prompt)
    document_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

    response = retrieval_chain.invoke({
        "chat_history": chat_history,
        "input": query
    })

    return response

def print_response(response, time):
    input = response["input"]
    context = response["context"]
    context = '\n\n'.join([f"[{idx}] {doc.page_content}" for idx, doc in enumerate(context, 1)])
    answer = response["answer"]

    print(f"context: {context}")
    print(f"question: {input}")
    print(f"answer: {answer}({time:.2f} sec)")
    return input, context, answer, time


# df = pd.DataFrame(columns=["model", "question", "answer", "context", "time"])
# df.loc[len(df)] = ["1", "2", "3", "4", "5"]
# df.to_csv("result.csv", sep="@")

for items in metrics:
    label = items.get('label')
    metric_func = items.get('value')
    # 검증 metrics
    for llm in llms:
        results = []
        llm_name = llm.name
        
        if llm_name is None:
            llm_name = "GPT-3.5"

        print('llm_name', llm_name)
        print("=" * 50)
        for idx, query in enumerate(queries):
            start_time = time.time()
            print('strated', idx)

            response = chat_with_llm(
                llm,
                vectorstore=vectorstore,
                prompt=prompt,
                query=query
            )
            input = response["input"]
            # context = '\n\n'.join([f"[{idx}] {doc.page_content}" for idx, doc in enumerate(response["context"], 1)])
            context = [f"[{idx}] {doc.page_content}" for idx, doc in enumerate(response["context"], 1)]
            answer = response["answer"]

            execution_time = time.time() - start_time
            print('input', input)
            # input, context, answer, _time = print_response(response, execution_time)
            print("-" * 50)
            print('context', context)
            print("-" * 50)
            print('answer', answer)
            print("-" * 50)

            test_result = metric_func(llm_name=llm_name, query=query, answer=answer, expected_output=answers[idx], context=context)
            results.append(test_result)
            print('inserted')
            # df.loc[len(df)] = [llm_name, input, answer, context, _time]
        # csv 저장
        pd.DataFrame(results).to_csv(f'{llm_name}_{label}_result.csv', encoding='utf-8-sig')
        print()

# df.to_csv("result.csv", sep="@")